# $(3,10)$--Matriods

In this notebook, we provide the computations used in the proof of Proposition 4.4, which asserts that $\operatorname{Gr}(\mathsf{Q};\mathbb{C})$ is smooth for all $\mathbb{C}$--realizable $(3,10)$--matroids. 

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
cd("..")

In [3]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))

realization_space_2_singular_locus (generic function with 1 method)

First we isolate those that satisfy the three lines property, i.e., every element of the ground set is contained in at least 3 lines (cyclic flats of rank 2). There are **151** such matroids.

In [ ]:
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d3n10 = Polymake.Polydb.find(collection, Dict("RANK" => 3, "SIMPLE"=>true, "N_ELEMENTS"=>10));


n3C10 = subsets(collect(1:10), 3);
n3C10 = sort(n3C10, by =  x-> reverse(x));

lines_3 = []
for c in d3n10
    Q = Matroid(c)
    ns = count_3_lines_thru_all_points(Q)
    if length(ns) == 0
        continue
    end    
    if minimum(ns) >= 3
        push!(lines_3, to_revlex(Q, n3C10))
    end
end

This data is precomputed in `d3n10/3lines_d3n10.dat`.

In [4]:
lines_3_precomputed = vec(readlines(joinpath(currentDir, "d3n10/data/3lines_3_10.dat")));

In [ ]:
Set(lines_3) == Set(lines_3_precomputed)

The file `d3n10/not_realizable.dat` contains those matroids in `d3n10/3lines_3_10.dat` that are not realizable over $\mathbb{Q}$. There are **44** such matroids.

In [5]:
not_realizable = vec(readlines(joinpath(currentDir, "d3n10/data/nonrealizable_3_10.dat")));
length(not_realizable)

44

In [6]:
test_nonrealizable = []

for Qstr in not_realizable
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [7]:
all(test_nonrealizable)

true

The files `zero_ideal_3_10.dat`, `univariate_ideal_3_10.dat`, and `multivariate_principal_ideal_3_10.dat` in `d3n10/data/` record matroids and a maximal circuit. A pair of matroid and circuit is recorded as a `*/0` vector. The first 10 entries record the circuit (e.g., `**0*00*000` is the circuit `[1,2,4,7]`) and the rest record the matroid in the usual revlex basis encoding. 

**Note:** One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`. To make this computation more transparent, we carry out the smoothness check explicitly.

The files `d3n10/zero_ideal_3_10.dat` contains those matroids in `d3n10/3lines_3_10.dat` and a maximal circuit such that the ideal of the realization space is zero. Therefore these realization spaces are smooth. There are **54** such matroids.

In [8]:
zero_ideal = vec(readlines(joinpath(currentDir, "d3n10/data/zero_ideal_3_10.dat")));
length(zero_ideal)

54

In [9]:
test_zero = []
for cir_Qstr in zero_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [10]:
all(test_zero)

true

The files `d3n10/data/univariate_ideal_3_10.dat` contains those matroids in `d3n10/data/3lines_3_10.dat` and a maximal circuit such that the ideal of the realization space is univariate. These realization spaces are smooth provided the discriminant is nonzero.  There are **37** such matroids.

In [11]:
univariate_ideal = vec(readlines(joinpath(currentDir, "d3n10/data/univariate_ideal_3_10.dat")));
length(univariate_ideal)

37

If $I\subset \mathbb{C}[x_1,\ldots,x_k]$ is an ideal principally generated by a polynomial in a single variable $x_1$, then $I$ defines a smooth scheme if and only if $f$  and $df/dx$ are relatively prime.  

In [12]:
test_univariate = []
for cir_Qstr in univariate_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    vs = ideal_vars(gens(I))
    length(vs) != 1 && error("not univariate")
    f = gens(I)[1]; #println("f = ", f); 
    df = derivative(f, vs[1]); #println("df = ", df); 
    push!(test_univariate, isone(ideal(MR.ambient_ring, [f,df])))
end

In [13]:
all(test_univariate)

true

The files `d3n10/multivariate_principal_ideal_3_10.dat` contains those matroids in `d3n10/3lines_3_10.dat` and a maximal circuit such that the ideal of the realization space is principal. There are **16** such matroids. By the Jacobian criterion, the affine scheme $\mathsf{Spec}(S^{-1}\mathbb{Q}[x_1,\ldots,x_n] / \langle f \rangle)$ is smooth if and only if the ideal

$$
J = \langle f, \frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n}\rangle
$$

is the unit ideal in $S^{-1} \mathbb{Q}[x_1,\ldots,x_n]$. This is equivalent to the saturation $(J:S^{\infty})$ being the unit ideal in $\mathbb{Q}[x_1,\ldots,x_n]$. We perform this explicit check here. 

In [14]:
principal_ideal = vec(readlines(joinpath(currentDir, "d3n10/data/multivariate_principal_ideal_3_10.dat")));
length(principal_ideal)

16

In [15]:
test_principal = []
for cir_Qstr in principal_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_principal, isone(Sing))
end

In [16]:
all(test_principal)

true

## Example 4.5

Let $\mathsf{Q}$ be the simple $(3,10)$-matroid whose lines are


$$
\mathcal{L}(\mathsf{Q}) = \{\{1,2,5\}, \{1,3,6\}, \{1,4,8\}, \{2,3,7\}, \{2,4,9\}, \{2,6,10\}, \{3,4,5\}, \{4,6,7\}, \{5,9,10\},\{6,8,9\}, \{7,8,10\}\}
$$



In [17]:
NB = [[1,2,5], [1,3,6], [1,4,8], [2,3,7], [2,4,9], [2,6,10], [3,4,5], [4,6,7], [5,9,10], [6,8,9], [7,8,10]]
A = [1,2,3,4]
Q = matroid_from_nonbases(NB,10)

Matroid of rank 3 on 10 elements

In [18]:
MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)

[1   0   0   1   x1   x2    0   x4   x6   x8]
[0   1   0   1    1    0   x3   x5   x7   x9]
[0   0   1   1    0    1    1    1    1    1]

The representations of the matroid are parametrized by the matrix
in the Multivariate polynomial ring in 9 variables over QQ
within the vanishing set of the ideal
ideal(x7 + x8 - x9 - 1, x6 - 1, x5 - 1, x2 - x8, x1 - 1, x4*x8 - x4*x9 - x4 - x8^2 + x8*x9 + 1, x3*x8 - x3 - x8, x3*x4 - x3 - x4*x9, x4*x9^2 - x4*x9 - x4 + x8 + 1, x3*x9^2 - x3*x9 + x3 - x8*x9 + x8 - x9, x8^2*x9 - x8^2 - x8*x9^2 + x8*x9 - x9)
avoiding the zero loci of the polynomials
RingElem[x7 - x9, x6 - x8, x6*x9 - x7*x8, x6*x9 - x6 - x7*x8 + x7 + x8 - x9, x2*x7 - x2*x9 + x6*x9 - x7*x8, x3*x6 - x3*x8 - x6*x9 + x7*x8, x4*x7 - x4*x9 - x5*x6 + x5*x8 + x6*x9 - x7*x8, x5 - x9, x4 - x8, x4*x9 - x5*x8  …  x1*x5 + x2 - x4, x1*x5 - x4, x1*x5 - x1 - x4 + 1, x4 - 1, x4 - x5, x1*x3 + x2, x1*x3 - x1 + 1, x3 - 1, x1 + x2 - 1, x2 - 1]


In [19]:
MR2 = reduce_ideal_full(MR)

[1   0   0   1   1   x1        0   x1^2 - x1*x2 - 1              1   x1]
[0   1   0   1   1    0       x1        x1 - x2 - 1   -x1 + x2 + 1   x2]
[0   0   1   1   0    1   x1 - 1        x1 - x2 - 1              1    1]

The representations of the matroid are parametrized by the matrix
in the Multivariate polynomial ring in 2 variables over QQ
within the vanishing set of the ideal
ideal(x1^2*x2 - x1^2 - x1*x2^2 + x1*x2 - x2)
avoiding the zero loci of the polynomials
RingElem[x1 - 1, x1 - x2, x2, x1^2 - x1*x2 + x2, x2 - 1, x1*x2 - x1 + 1, x1*x2 - x1 - x2, x1, x1^2 - x1*x2 - x1 + x2 + 1, x1 - x2 - 1, x1^2 - x1*x2 - 1, x1^3 - 2*x1^2 - x1*x2^2 + 2*x1*x2 - 2*x2]


In [20]:
f = MR2.defining_ideal[1]
R = MR2.ambient_ring
x1,x2 = gens(R)

2-element Vector{QQMPolyRingElem}:
 x1
 x2

In [21]:
factor(derivative(f,x1))

1 * (x2 - 1) * (2*x1 - x2)

In [22]:
factor(derivative(f,x2))

1 * (x1^2 - 2*x1*x2 + x1 - 1)

In [23]:
isone(ideal([f, 2*x1-x2, x1^2-2*x1*x2+x1-1]))

true